# Churn analysis

В этом задании предлагается проанализировать данные одной из американских телекоммуникационных компаний о пользователях, которые потенциально могут уйти.

Измерены следующие признаки:

* state — штат США
* account_length — длительность использования аккаунта
* area_code — деление пользователей на псевдорегионы, использующееся в телекоме
* intl_plan — подключена ли у пользователя услуга международного общения
* vmail_plan — подключена ли у пользователя услуга голосовых сообщений
* vmail_message — количество голосых сообщений, который пользователь отправил / принял
* day_calls — сколько пользователь совершил дневных звонков
* day_mins — сколько пользователь проговорил минут в течение дня
* day_charge — сколько пользователь заплатил за свою дневную активность
* eve_calls, eve_mins, eve_charge — аналогичные метрики относительно вечерней активности
* night_calls, night_mins, night_charge — аналогичные метрики относительно ночной активности
* intl_calls, intl_mins, intl_charge — аналогичные метрики относительно международного общения
* custserv_calls — сколько раз пользователь позвонил в службу поддержки
* treatment — номер стратегии, которая применялись для удержания абонентов (0, 2 = два разных типа воздействия, 1 = контрольная группа)
* mes_estim — оценка интенсивности пользования интернет мессенджерами
* *churn* — результат оттока: перестал ли абонент пользоваться услугами оператора

In [12]:
data=pd.read_csv('churn_analysis.csv')

In [13]:
data.drop(columns=u'Unnamed: 0',axis=1,inplace=True)

In [50]:
data[0:10]

,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,OH,107,415,no,yes,26,161.6,123,27.47,195.5,...,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,NJ,137,415,no,no,0,243.4,114,41.38,121.2,...,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,OK,75,415,yes,no,0,166.7,113,28.34,148.3,...,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.
5,AL,118,510,yes,no,0,223.4,98,37.98,220.6,...,203.9,118,9.18,6.3,6,1.70,0,2,0.60,False.
6,MA,121,510,no,yes,24,218.2,88,37.09,348.5,...,212.6,118,9.57,7.5,7,2.03,3,0,0.28,False.
7,MO,147,415,yes,no,0,157.0,79,26.69,103.1,...,211.8,96,9.53,7.1,6,1.92,0,0,0.59,False.
8,LA,117,408,no,no,0,184.5,97,31.37,351.6,...,215.8,90,9.71,8.7,4,2.35,1,1,0.50,False.
9,WV,141,415,yes,yes,37,258.6,84,43.96,222.0,...,326.4,97,14.69,11.2,5,3.02,0,2,0.45,False.


Давайте рассмотрим всех пользователей из контрольной группы (treatment = 1). Для таких пользователей мы хотим проверить гипотезу о том, что штат абонента не влияет на то, перестанет ли абонент пользоваться услугами оператора.

Для этого мы воспользуемся критерием хи-квадрат. Построим таблицы сопряженности между каждой из всех 1275 возможных неупорядоченных пар штатов и значением признака churn. Для каждой такой таблицы 2x2 применить критерий хи-квадрат можно с помощью функции scipy.stats.chi2_contingency(subtable, correction=False).

Заметьте, что, например, (AZ, HI) и (HI, AZ) — это одна и та же пара. Обязательно выставим correction=False (о том, что это значит мы узнаем далее).

Сколько достигаемых уровней значимости оказались меньше, чем α=0.05?

In [54]:
st=pd.pivot_table(data.where(data.treatment==1).dropna(),index='state',columns='churn',values='treatment',aggfunc=len,fill_value=0)


In [59]:
corr_data=[]

for i, lhs_column in enumerate(st.columns):
    for j, rhs_column in enumerate(st.columns):
        if i>=j:
            continue
        corr,hi=sts.chi2_contingency((st[lhs_column],st[rhs_column]),correction=False)
        corr_data.append([lhs_column,rhs_column,corr,p])

ValueError: too many values to unpack

In [ ]:
ste=pd.pivot_table()